### Шаг 1. Изучение общей информации

In [1]:
import pandas as pd
import numpy as np
import plotly as plt
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats as st

ModuleNotFoundError: No module named 'plotly'

In [ ]:
df = pd.read_csv('/datasets/games.csv')

In [ ]:
print(df.info())
display(df.head(10))

for col in df.columns: #посчитаем процент пропусков для наглядности
    pct_missing = np.mean(df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

#### Вывод

Всего в таблице с информацией об играх 11 столбцов, 6 принадлежат типу float, 5 - object. Названия стобцов некорректны (разный регистр), в данных есть пропуски. Пропуски в данных о рейтинге критиков, пользователей и ассоциации игр могут свидетельствовать о том, что рейтинга игры нет в открытом доступе. Пропуски в данных о годе релиза и названия игры могут быть случайными.

Не все типы данных корректны - данные столбца с годом релиза принадлежит типу числа с плавающей точкой (больше подходит целочисленный тип), данные столбца user_score принадлежат категориальному типу(больше подходит тип числа с плавающей точкой).


### Шаг 2. Подготовка данных

#### Замена названий столбцов 

In [ ]:
df.columns = df.columns.str.lower() #приведение названий к нижнему регистру
df.columns #проверка

#### Обработка пропусков и замена типа данных

In [ ]:
df.dropna(subset=['name'], inplace=True) #удаляем пропущенные значения

In [ ]:
df.dropna(subset=['year_of_release'], inplace=True) #удаляем пропущенные значения
df['year_of_release'] = df['year_of_release'].astype('int') #меняем тип данных


In [ ]:
df.loc[df['user_score']=='tbd','user_score']='NaN' #заменяем значения tbd на NaN
df['user_score']=df['user_score'].fillna('NaN') #заменяем nan на NaN, чтобы потом изменить тип данных
df['user_score'] = df['user_score'].astype('float') #меняем тип данных

In [ ]:
print('Процент нулевых значений в данных о продажах США: {:.1%}'.format(df.loc[df['na_sales']==0,'na_sales'].count()/len(df['na_sales'])))
print('Процент нулевых значений в данных о продажах Европа: {:.1%}'.format(df.loc[df['eu_sales']==0,'eu_sales'].count()/len(df['eu_sales'])))
print('Процент нулевых значений в данных о продажах Япония: {:.1%}'.format(df.loc[df['jp_sales']==0,'jp_sales'].count()/len(df['jp_sales'])))
print('Процент нулевых значений в данных о продажах другие страны: {:.1%}'.format(df.loc[df['other_sales']==0,'other_sales'].count()/len(df['other_sales'])))


In [ ]:
df['general_sales'] = df['na_sales'] + df['eu_sales'] + df['jp_sales'] + df['other_sales']

In [ ]:
df.info() #проверка

#### Вывод

Пропусков в данных с годом релиза немного - всего 2% от всех данных. Они были удалены, т.к.  их отсутствие не должно сильно повлиять на дальнейший анализ. 
Также были удалены пропуски с названием игры, т.к. их меньше 1%.

Значения tbd в данных с рейтингом по сути представляют собой пропущенные значения, поэтому приводим их к NaN.
Пропусков в рейтингах очень много - от 40 до 51%. Их не стоит трогать, т.к. их удаление повлияет на все данные. 
Был изменён тип данных с категориального на float.

Также в данных о продажах много нулевых значений - они могут быть связаны как с тем, что в некоторых регионах конкретная игра не продавалась, так и с тем, что в открытом доступе не было соответствующих данных. Их тоже трогать не будем, чтобы не повлиять на весь датасет.

Была подсчитана общая сумма продаж в новый столбец - general_sales.

### Шаг 3. Исследовательский анализ данных

#### 1. Сколько игр выпускалось в разные годы

In [ ]:
def hist(data, column, title, x_title, y_title): #функция вывода гистограммы
    fig = px.histogram(
    data,
    x = column,
    color_discrete_sequence=['#00CC96'])
    fig.update_layout(title = title,
                      xaxis_title = x_title,
                      yaxis_title = y_title)
    print(data[column].describe())
    fig.show()

In [ ]:
hist(df, 'year_of_release', 'Распределение количества игр по годам выпуска', "Год выпуска", "Количество игр")

#### Вывод

Распределение близко к нормальному. Начиная с 1991 г. количество выпускаемых игр начинает расти, что в целом логично, ведь начинает развиваться индустрия копьютерных игр. Резкий скачок наблюдается в 2002 г. - за год выпускается в 1.7 раза больше игр, чем в предыдущем году. Следующий скачок происходит в 2005 - количество выпускаемых игр растет вплоть до 2008-2009 года (рост выходит на плато) и далее количество выпускаемых игр снижается. Это может быть связано с тем, что игр становится очень много и сейчас многие компании работают не только над выпуском новых, но и над обновлениями старых игр.

В связи со снижением значений количества игр имеет смысл рассматривать данные, начиная с 2012-13 года, ведь мы в целом наблюдаем тенденцию к резкому уменьшений количества выпускаемых игр.

#### 2. Продажи по платформам

In [ ]:
df_piv_platform = df.pivot_table(index='platform',  values ='general_sales', aggfunc = ['sum']).reset_index()
df_piv_platform.columns = ['Платформа','Суммарные продажи']
df_piv_platform = df_piv_platform.sort_values(by='Суммарные продажи', ascending=False, inplace = False).reset_index(drop=True)
display(df_piv_platform)

fig = px.histogram(df, x="platform", y="general_sales", histfunc='sum')
fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig.show()

#### Вывод

Суммарные продажи большей части платформ превышают 15 млн. Выделяются явные лидеры - продажи платформ PS2, X360, PS3, Wii, DC и PC составили от 727 млн до 123 млн.

7 платформ выделяются сравнительно низкими суммами продаж - меньше 2 миллионов. Удалим эти платформы, т.к. их данные могут исказить результаты исследования.

In [ ]:
df_platform = df_piv_platform[df_piv_platform["Суммарные продажи"] > 15] 
platfrom_columns = df_platform['Платформа']
df_new = df[df.platform.isin(platfrom_columns)] #создаём новый df с продажами больше 15 млн.

new_data = df_new.groupby(['year_of_release','platform'], as_index=False).agg({'name':'count'}) #группируем таблицу по платформе, году релиза и количеству игр
                                                                                        

In [ ]:
def line(data, x_column, y_column, color_column, title, x_title, y_title):
    fig = px.line(data, x = x_column, y= y_column, color = color_column) #строим линейный график 
    fig.update_layout(title = title,
                      xaxis_title = x_title,
                      yaxis_title = y_title)
    fig.show()

In [ ]:
line(new_data, 'year_of_release', 'name', 'platform', 'Распределение продаж платформ по годам', 'Год', 'Количество продаж')    


#### Вывод

На графике мы наблюдаем срок существования платформ: часть платформ уже не выпускают игры, по ним мы можем проследить средний срок существования платформы - 10 лет. Меньше всего существовала платформа GB (3 года), больше всего - платформы PC и DC (31, 28 лет). 

Исходя из полученных данных, чтобы спрогнозировать продажи на 2017 год, мы можем использовать данные, начиная с 2013 г. - в этом году также появляются 2 новые перспективные платформы XOne и PS4 - количество выпускаемых на них игр растёт. Это также соотносится с нашим выводом относительно общего количества выпускаемых игр - в 2012-13 годах по графику мы наблюдали резкое падение.

Таким образом, в дальнейшем исследовании будут представлены игры, выпускаемые с 2013 года.

#### 3. Исследование продаж актуального периода

In [ ]:
df_2017 = df_new.query('year_of_release >= 2013').reset_index(drop=True) #выделяем игры, выпущенные с 2013 г.
df_2017.drop(df_2017[df_2017['platform'] == 'DS'].index, inplace=True) #удаляем платформы DS, т.к. в 2013 она прекратила существование

In [ ]:
platform_2017 = df_2017.groupby(['year_of_release','platform'], as_index=False).agg({'general_sales':'sum'})

line(platform_2017, 'year_of_release', 'general_sales', 'platform', 'Распределение суммы продаж платформ по годам с 2013', 'Год', 'Сумма продаж')  


#### Вывод

На графике распределения суммы продаж платформ по годам мы можем увидеть лидирующие по продажам платформы: XOne и PS4. Эти платформы находились на пике популярности в 2015 году, сейчас продажи постепенно уменьшаются, но на 2016 г. всё еще остаются высокими. Высокими продажами на 2016 г. также выделяется платформа 3DS.

Глобальные продажи оставшихся платформ стремительно снижаются.

#### 4. Боксплоты по  глобальным продажам

In [ ]:
def box(data, x_column, y_column, title, x_title, y_title): #функция вывода ящика с усами
    fig = px.box(data, x=x_column, y=y_column)
    fig.update_layout(barmode='stack', title=title, xaxis_title=x_title, yaxis_title=y_title)
    fig.show()

In [ ]:
box(df_2017, 'platform', 'general_sales','Глобальные продажи по платформам', 'Продажи', 'Платформы')


#### Вывод

Практически все платформы имеют выбросы в данных, после 5 млн значения очень редки, их можно отрезать для дальнейшего анализа. Самыми высокими медианными значениями по продажам выделяются платформы X360, XOne, PS3, PS4 и WiiU - их можно назвать самыми популярными.

#### 5. Зависимость продаж от отзывов критиков

In [ ]:
df_PS4 = df_2017.query("platform == 'PS4' & general_sales < 5").reset_index(drop=True)


print("Корреляция продаж и рейтинга от пользователей и критиков PS4:")
print(df_PS4[['critic_score', 'user_score']].corrwith(df_PS4['general_sales']))
columns_score = ['critic_score', 'user_score', 'general_sales']
fig = px.scatter_matrix(df_PS4[columns_score], title = 'Диаграмма матрицы рассеивания продаж и рейтинга PS4')
fig.show()

In [ ]:
df_other = df_2017.query("platform != 'PS4' & general_sales < 5").reset_index(drop=True)
print("Корреляция продаж и рейтинга от пользователей и критиков других игр:")
print(df_other[['critic_score', 'user_score']].corrwith(df_other['general_sales']))
fig = px.scatter_matrix(df_other[columns_score], title = 'Диаграмма матрицы рассеивания продаж и рейтинга других игр')
fig.show()

#### Вывод

По результатам корреляции Пирсона мы наблюдаем прямую взаимосвязь между оценкой критиков игры PS4 и её продажами. Рейтинг критиков других игр также коррелирует с их продажами, но в меньшей степени. Взаимосвязи между оценкой пользователей и продажами не обнаружена, коэфициент корреляции близок к нулю. Данные выводы иллюстрируются диаграммами на матрице рассеяния.

#### 6. Распределение игр по жанрам

In [ ]:
fig = px.histogram(
df_2017,
x = 'genre',
color_discrete_sequence=['#00CC96'])
fig.update_layout(title = 'Распределение игр по жанру', 
                  xaxis_title="Жанр",
                  yaxis_title="Количество игр жанра",
                  xaxis={'categoryorder':'total descending'})
print(df_2017['genre'].describe())
fig.show()

In [ ]:
fig = px.histogram(df_2017, x="genre", y="general_sales", histfunc='sum')
fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'},
                  title = 'Распределение жанров по сумме продаж', 
                  xaxis_title="Жанр",
                  yaxis_title="Сумма продаж")
fig.show()

#### Вывод

Среди количества выпущенных игр особой популярностью выделяется жанр action, он также является самым прибыльным. Также высокую прибыль приносят жанры shooter, sport и role-playing. Жанр adventure также отличается большим количеством выпущенных игр, при том что прибыль от него не высока. Игры жанра simulation, strategy и puzzle выпускаются редко и отличаются низкими продажами.

### Шаг 4. Составление портрета пользователя каждого региона

In [ ]:
top5_sales_na = df_2017.pivot_table(index='platform',  values ='na_sales', aggfunc ='sum').reset_index().sort_values(by = 'na_sales', ascending=False).head()
top5_sales_eu = df_2017.pivot_table(index='platform',  values ='eu_sales', aggfunc ='sum').reset_index().sort_values(by = 'eu_sales', ascending=False).head()
top5_sales_jp = df_2017.pivot_table(index='platform',  values ='jp_sales', aggfunc ='sum').reset_index().sort_values(by = 'jp_sales', ascending=False).head()


In [ ]:
def pie(dataframe, column_labels, column_values, title): #функция вывода круговой диаграммы
    labels = dataframe[column_labels]
    values = dataframe[column_values]

    fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='label+percent')])
    fig.update_layout(title = title)
    fig.show()

In [ ]:
pie(top5_sales_na, 'platform', 'na_sales', 'Доля продаж популярных игр в Северной Америке')
pie(top5_sales_eu, 'platform', 'eu_sales', 'Доля продаж популярных игр в Европе')
pie(top5_sales_jp, 'platform', 'jp_sales', 'Доля продаж популярных игр в Японии')



#### Вывод

Топ 5 платформ в Северной Америке и Европе одинаков - это платформы PS4, PS3, XOne, X360 и 3DS.
Самая популярная платформа: PS4, при этом в Европе по доле продаж она занимает 42,2% и является явным лидером. В США она занимает долю в 28,2%. 
Следующие по популярности платформы в Северной Америке - XOne (24,2%), X360(21,2%), PS3(16,5%), в Европе - PS3(20,3%), XOne(15,4%), X360(12,7%). 
Закрывает рейтинг платформа 3DS: её доля продаж в Северной Америке 9,92%, а в Европе 9,27%. 
Рейтинг в Японии отличается. Там самая популярная платформа 3DS (49.6%), далее идут платформы PS3 (17,1), PSV (13,6), PS4(11,7), WiiU(7,97).  
Общие популярные платформы в трёх регионах - PS3 и 3DS.



In [ ]:
genres_sales_na = df_2017.pivot_table(index='genre',  values ='na_sales', aggfunc ='sum').reset_index().sort_values(by = 'na_sales', ascending=False).head()
genres_sales_eu = df_2017.pivot_table(index='genre',  values ='eu_sales', aggfunc ='sum').reset_index().sort_values(by = 'eu_sales', ascending=False).head()
genres_sales_jp = df_2017.pivot_table(index='genre',  values ='jp_sales', aggfunc ='sum').reset_index().sort_values(by = 'jp_sales', ascending=False).head()

In [ ]:
pie(genres_sales_na, 'genre', 'na_sales', 'Доли продаж популярных жанров в Северной Америке')
pie(genres_sales_eu, 'genre', 'eu_sales', 'Доли продаж популярных жанров в Европе')
pie(genres_sales_jp, 'genre', 'jp_sales', 'Доли продаж популярных жанров в Японии')

#### Вывод

Топ 4 популярных жанров в Европе и Северной Америке сходятся: 1 место занимает жанр action (Северная Америка - 33,6%, Европа - 36,5%), 2 - shooter (Северная Америка - 29,3%, Европа - 27,2%), 3 - sports (Северная Америка - 17,4%, Европа - 18,7%), 4 - role-playing(Северная Америка - 12,4%, Европа - 11,4%). Закрывает рейтинг в Европе жанр racing (6,24%), в Северной Америке - Misc (7,33%).

Данные в Японии вновь отличаются. Явные лидеры там жанры Role-Playing(44,4%) и Action(35,2%), т.е. на два этих жанра приходится доля в 79%. Далее следуют жанры Misc(8%), Fighting(6,65%) и shooter(5,75%).

Общие популярные жанры 3 регионов; action, shooter, role-playing.


In [ ]:
rating_sales = df_2017.pivot_table(index='rating',  values =['na_sales', 'eu_sales', 'jp_sales'], aggfunc ='sum').reset_index()

In [ ]:
pie(rating_sales, 'rating', 'na_sales', 'Доли продаж популярных жанров в Северной Америке')
pie(rating_sales, 'rating', 'eu_sales', 'Доли продаж популярных жанров в Европе')
pie(rating_sales, 'rating', 'jp_sales', 'Доли продаж популярных жанров в Японии')

#### Вывод

Распределение долей по рейтингу в Европе и Северной Америке схоже - самым распространённым является рейтинг M - для взрослых (доля в Северной Америке - 47,5 %, в Европе - 46,5%), далее идут рейтинги: E - для всех (22,7% в Северной Америке, 26,5% в Европе), e10 - для детей от 10 лет (15,5% в Северной Америке, 13,6% в Европе), T - для подростков (14,3% в Северной Америке, 13,4% в Европе).

В Японии самыми распространёнными являются игры с рейтингом для подростков (36,9%), далее следуют рейтинги: для всех (27,2%), для взрослых (25,3%), для детей от 10 лет (10,6%).

### Шаг 5. Проверка гипотез

#### 1 гипотеза.

Нулевая гипотеза - средние пользовательские рейтинги платформ Xbox One и PC одинаковые. Альтернативная гипотеза - средний пользовательские рейтинги платформ Xbox One и PC отличаются.

In [ ]:
df_XOne = df_2017.query('platform == "XOne" & user_score != "NaN"')
df_PC = df_2017.query('platform == "PC" & user_score != "NaN"')


variance_XOne = np.var(df_XOne['user_score'], ddof=1)
variance_PC = np.var(df_PC['user_score'], ddof=1)

print('Дисперсия пользовательского рейтинга XOne', variance_XOne)
print('Дисперсия пользовательского рейтинга PC', variance_PC)
print('Процент разницы: {:.1%}'.format(1 - variance_XOne/variance_PC))
print()

alpha = .05

results = st.ttest_ind(
    df_XOne['user_score'], 
    df_PC['user_score'],
    equal_var = False)

print('p-значение:', results.pvalue)

if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу") 
  

#### Вывод

Гипотеза не отклоняется, можем сделать вывод, что средний пользовательский рейтинг платформы PC вероятно не отличается от среднего рейтинга платформы XOne.

#### 2 гипотеза.

Нулевая гипотеза - cредние пользовательские рейтинги жанров Action и Sports одинаковы. Альтернативная - средние пользовательские рейтинги жанров Action и Sports разные.

In [ ]:
df_action = df_2017.query('genre == "Action" & user_score != "NaN"')
df_sports = df_2017.query('genre == "Sports" & user_score != "NaN"')

variance_action = np.var(df_action['user_score'], ddof=1)
variance_sports = np.var(df_sports['user_score'], ddof=1)

print('Дисперсия пользовательского рейтинга XOne', variance_action)
print('Дисперсия пользовательского рейтинга PC', variance_sports)
print('Процент разницы: {:.1%}'.format(1 - variance_action/variance_sports))
print()

alpha = .05

results = st.ttest_ind(
    df_action['user_score'], 
    df_sports['user_score'],
    equal_var = False)

print('p-значение:', results.pvalue)

if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу") 

#### Вывод

Нулевая гипотеза ложна, её можно отклонить - т.е. средние пользовательские рейтинги жанров Action и Sports разные.

### Шаг 6. Общий вывод

Самыми высокими продажами по данным с 1980 года по 2016 выделяются платформы PS3, X360, PS3, Wii, DC, PC. Средний срок существования платформ – 10 лет.

Актуальным для составления прогноза на 2017 год был выбран период с 2013 г., в этом году появились 2 новые перспективные платформы – XOne и PS4.

Опираясь на график распределения суммы продаж платформ по годам, мы можем спрогнозировать, что самые высокие продажи в 2017 году принесут платформы XOne, PS4 и 3DS. Популярными также останутся платформы X360, PS3 и WiiU.

Прогнозируя продажи, также важно учитывать рейтинг критиков – чем он выше, тем больше сумма продаж.

Самый популярный и прибыльный жанр среди игр – action. Также высокими продажами отличаются жанры shooter, sport и role-playing. На высокие продажи жанров simulation, strategy и puzzle можно не рассчитывать. 

Данные о популярности платформ, жанров и рейтингов в Северной Америке и Европе схожи и отличаются от данных Японии, это следует учитывать при настройке рекламных кампаний.

В Северной Америке и Европе самыми популярными являются платформы: PS4, XOne, X360, PS3, 3DS. Общий топ жанров: action, shooter, sports, role-playing. Также популярны жанры Misc в Северной Америке и Racing в Европе. Самый распространённый рейтинг M, далее идут рейтинги E, E10, T.  

В Японии самые популярные платформы 3DS, PS3, PSV, PS4, WiiU. Жанры: role-playing, action, misc, fighting, shooter.  Самый распространённый рейтинг T, далее идут рейтинги E, M, E10. 
